# Extract Data

In [2]:
root_path = '/home/santosh/Work/Datasets/CORD-19-research-challenge/'
paths = [
    'biorxiv_medrxiv/biorxiv_medrxiv/',
        'comm_use_subset/comm_use_subset/',
        'noncomm_use_subset/noncomm_use_subset/'
        ]

result_path = '/home/santosh/Work/Datasets/CORD-19-sentences/'



In [3]:
import glob
import json
from tqdm import tqdm
import pandas as pd

In [4]:
# get all the jsosn

all_jsons =[]
for json_path in paths:
     all_jsons.extend(sorted(glob.glob(root_path+json_path + '*.json*')))

# all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)

In [5]:
all_jsons[0]

'/home/santosh/Work/Datasets/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/0015023cc06b5362d332b3baf348d11567ca2fbb.json'

In [6]:
meta_df = pd.read_csv('/home/santosh/Work/Datasets/CORD-19-research-challenge/metadata.csv')

In [7]:
meta_df.columns

Index(['sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id', 'license',
       'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_full_text',
       'full_text_file'],
      dtype='object')

In [8]:
selected_columns = ['sha', 'doi', 'pmcid', 'pubmed_id', 'publish_time'] # sha is paper id here

In [9]:
meta_ref_df = meta_df[selected_columns]
meta_ref_df = meta_ref_df[meta_ref_df['sha'].notna()]


In [10]:
# extract sentences with doi


for each_json_file in tqdm(all_jsons):
    with open(each_json_file) as json_file:
        data = json.load(json_file)
        
        try:
            source = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['doi'].values.tolist()[0]
        except:
            source =''
            
        full_text = []

        try:
            for each_text in data['metadata']:
                full_text.append(each_text['title']+' ---'+source+'---')
        except:
                pass 

        
        try:
            for each_text in data['abstract']:
                full_text.append(each_text['text']+' ---'+source+'---')
        except:
                pass

        try:
            for each_text in data['body_text']:
                full_text.append(each_text['text']+' ---'+source+'---')
        except:
                pass
            
        try:
            for each_text in data['ref_entries']:
                full_text.append(data['ref_entries'][each_text]['text']+' ---'+source+'---')
        except:
                pass    
        
        with open(result_path+each_json_file.split('/')[-1][:-5]+'.txt', 'a') as writer:
            for each_para in full_text:
                    if 'doi:' not in each_para and 'word count' not in each_para and 'All rights reserved' not in each_para and 'No reuse allowed without permission' not in each_para:
                        writer.write(each_para+'\n')     

100%|██████████| 12356/12356 [03:33<00:00, 57.96it/s]


# Generate Fasttext

In [2]:
# Here I am using gensim for learning the Fast Text word embeddings from the CORD-19
import nltk
import logging
import os
# download NLTK basic stopwords
import nltk
from nltk.corpus import stopwords

from nltk.tokenize.treebank import TreebankWordDetokenizer, TreebankWordTokenizer


# start the log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)


def clean_my_text(full_text_):
    stopset = set(stopwords.words('english')) #| set(string.punctuation)
    tokens = TreebankWordTokenizer().tokenize(full_text_.split('---10')[0])
    cleanup = [token for token in tokens if token not in stopset and len(token) > 1]
    return cleanup


# get the current working directory and file
data_dir_path = '/home/santosh/Work/Datasets/CORD-19-sentences/'
result_dir_path = '/home/santosh/Work/models/word2vec/CORD-19/'

In [3]:
class SentenceClass(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
#             print(fname)
            for line in open(os.path.join(self.dirname, fname), 'r'):
                yield clean_my_text(line)


# iterate one line at a time and learn embeddings
paragraphs = SentenceClass(data_dir_path)  # a memory-friendly iterator

In [4]:
from gensim.models.fasttext import FastText

model = FastText(size=200,
    window=5,
    min_count=5,
    workers=4, iter=5)

model.build_vocab(sentences=paragraphs)
training_examples_count = model.corpus_count







2020-03-30 23:24:12,634 : INFO : resetting layer weights
2020-03-30 23:24:22,285 : INFO : collecting all words and their counts
2020-03-30 23:24:22,323 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-30 23:24:31,662 : INFO : PROGRESS: at sentence #10000, processed 808238 words, keeping 67684 word types
2020-03-30 23:24:41,977 : INFO : PROGRESS: at sentence #20000, processed 1624424 words, keeping 105701 word types
2020-03-30 23:24:50,885 : INFO : PROGRESS: at sentence #30000, processed 2429497 words, keeping 138800 word types
2020-03-30 23:25:00,007 : INFO : PROGRESS: at sentence #40000, processed 3263053 words, keeping 172853 word types
2020-03-30 23:25:11,423 : INFO : PROGRESS: at sentence #50000, processed 4053294 words, keeping 199081 word types
2020-03-30 23:25:20,401 : INFO : PROGRESS: at sentence #60000, processed 4908065 words, keeping 230538 word types
2020-03-30 23:25:29,350 : INFO : PROGRESS: at sentence #70000, processed 5745451 words, kee

In [5]:
model.train(paragraphs, total_examples=training_examples_count, epochs=model.iter)
logging.info("Done training data files")

/home/santosh/anaconda3/envs/scispacy/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.
2020-03-30 23:32:19,582 : INFO : training model with 4 workers on 196469 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-03-30 23:32:20,628 : INFO : EPOCH 1 - PROGRESS: at 0.16% examples, 64252 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:32:21,665 : INFO : EPOCH 1 - PROGRESS: at 0.40% examples, 78394 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:32:22,702 : INFO : EPOCH 1 - PROGRESS: at 0.66% examples, 85655 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:32:23,712 : INFO : EPOCH 1 - PROGRESS: at 0.92% examples, 87371 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:32:24,798 : INFO : EPOCH 1 - PROGRESS: at 1.20% examples, 89206 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:32:25,826 : INFO : EPOCH 1 - P

2020-03-30 23:33:34,134 : INFO : EPOCH 1 - PROGRESS: at 18.69% examples, 95705 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:35,225 : INFO : EPOCH 1 - PROGRESS: at 18.90% examples, 95885 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:36,294 : INFO : EPOCH 1 - PROGRESS: at 19.17% examples, 95920 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:37,308 : INFO : EPOCH 1 - PROGRESS: at 19.47% examples, 96016 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:38,326 : INFO : EPOCH 1 - PROGRESS: at 19.74% examples, 96104 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:39,396 : INFO : EPOCH 1 - PROGRESS: at 20.02% examples, 96129 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:40,399 : INFO : EPOCH 1 - PROGRESS: at 20.29% examples, 96232 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:41,448 : INFO : EPOCH 1 - PROGRESS: at 20.54% examples, 96275 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:33:42,527 : INFO : EPOCH 1 - PROGRESS: at 20.84% examples, 96276 words/s, in_qsize 0, out_

2020-03-30 23:34:52,315 : INFO : EPOCH 1 - PROGRESS: at 38.71% examples, 95772 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:53,333 : INFO : EPOCH 1 - PROGRESS: at 39.02% examples, 95755 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:54,384 : INFO : EPOCH 1 - PROGRESS: at 39.29% examples, 95782 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:55,475 : INFO : EPOCH 1 - PROGRESS: at 39.58% examples, 95785 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:56,490 : INFO : EPOCH 1 - PROGRESS: at 39.84% examples, 95835 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:57,554 : INFO : EPOCH 1 - PROGRESS: at 40.10% examples, 95852 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:58,609 : INFO : EPOCH 1 - PROGRESS: at 40.39% examples, 95875 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:34:59,690 : INFO : EPOCH 1 - PROGRESS: at 40.69% examples, 95883 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:35:00,773 : INFO : EPOCH 1 - PROGRESS: at 40.98% examples, 95883 words/s, in_qsize 0, out_

2020-03-30 23:36:09,898 : INFO : EPOCH 1 - PROGRESS: at 59.09% examples, 96631 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:10,965 : INFO : EPOCH 1 - PROGRESS: at 59.33% examples, 96596 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:12,000 : INFO : EPOCH 1 - PROGRESS: at 59.57% examples, 96536 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:13,078 : INFO : EPOCH 1 - PROGRESS: at 59.79% examples, 96499 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:14,182 : INFO : EPOCH 1 - PROGRESS: at 60.11% examples, 96489 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:15,208 : INFO : EPOCH 1 - PROGRESS: at 60.39% examples, 96514 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:16,224 : INFO : EPOCH 1 - PROGRESS: at 60.67% examples, 96542 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:17,263 : INFO : EPOCH 1 - PROGRESS: at 60.98% examples, 96563 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:36:18,287 : INFO : EPOCH 1 - PROGRESS: at 61.24% examples, 96587 words/s, in_qsize 0, out_

2020-03-30 23:37:27,244 : INFO : EPOCH 1 - PROGRESS: at 79.50% examples, 96867 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:28,348 : INFO : EPOCH 1 - PROGRESS: at 79.75% examples, 96861 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:29,432 : INFO : EPOCH 1 - PROGRESS: at 80.03% examples, 96860 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:30,477 : INFO : EPOCH 1 - PROGRESS: at 80.26% examples, 96811 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:31,537 : INFO : EPOCH 1 - PROGRESS: at 80.49% examples, 96788 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:32,587 : INFO : EPOCH 1 - PROGRESS: at 80.77% examples, 96766 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:33,636 : INFO : EPOCH 1 - PROGRESS: at 81.03% examples, 96778 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:34,641 : INFO : EPOCH 1 - PROGRESS: at 81.30% examples, 96802 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:37:35,648 : INFO : EPOCH 1 - PROGRESS: at 81.54% examples, 96825 words/s, in_qsize 0, out_

...

2020-03-30 23:57:36,831 : INFO : EPOCH 5 - PROGRESS: at 2.21% examples, 101373 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:37,887 : INFO : EPOCH 5 - PROGRESS: at 2.52% examples, 101190 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:38,908 : INFO : EPOCH 5 - PROGRESS: at 2.79% examples, 101359 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:39,923 : INFO : EPOCH 5 - PROGRESS: at 3.07% examples, 101540 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:41,013 : INFO : EPOCH 5 - PROGRESS: at 3.34% examples, 101735 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:42,050 : INFO : EPOCH 5 - PROGRESS: at 3.65% examples, 101705 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:43,073 : INFO : EPOCH 5 - PROGRESS: at 3.91% examples, 101763 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:44,145 : INFO : EPOCH 5 - PROGRESS: at 4.19% examples, 102084 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:57:45,199 : INFO : EPOCH 5 - PROGRESS: at 4.48% examples, 101921 words/s, in_qsize 0, out_

2020-03-30 23:58:52,926 : INFO : EPOCH 5 - PROGRESS: at 22.43% examples, 101657 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:58:53,990 : INFO : EPOCH 5 - PROGRESS: at 22.72% examples, 101615 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:58:55,044 : INFO : EPOCH 5 - PROGRESS: at 23.00% examples, 101573 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:58:56,050 : INFO : EPOCH 5 - PROGRESS: at 23.23% examples, 101487 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:58:57,085 : INFO : EPOCH 5 - PROGRESS: at 23.53% examples, 101385 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:58:58,156 : INFO : EPOCH 5 - PROGRESS: at 23.79% examples, 101239 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:58:59,191 : INFO : EPOCH 5 - PROGRESS: at 24.09% examples, 101124 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:59:00,285 : INFO : EPOCH 5 - PROGRESS: at 24.42% examples, 101067 words/s, in_qsize 0, out_qsize 0
2020-03-30 23:59:01,324 : INFO : EPOCH 5 - PROGRESS: at 24.70% examples, 101072 words/s, in_qsiz

2020-03-31 00:00:08,819 : INFO : EPOCH 5 - PROGRESS: at 42.77% examples, 100478 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:09,856 : INFO : EPOCH 5 - PROGRESS: at 43.08% examples, 100481 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:10,912 : INFO : EPOCH 5 - PROGRESS: at 43.38% examples, 100473 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:12,010 : INFO : EPOCH 5 - PROGRESS: at 43.67% examples, 100494 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:13,056 : INFO : EPOCH 5 - PROGRESS: at 43.98% examples, 100487 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:14,077 : INFO : EPOCH 5 - PROGRESS: at 44.25% examples, 100501 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:15,081 : INFO : EPOCH 5 - PROGRESS: at 44.53% examples, 100525 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:16,152 : INFO : EPOCH 5 - PROGRESS: at 44.80% examples, 100532 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:00:17,233 : INFO : EPOCH 5 - PROGRESS: at 45.07% examples, 100560 words/s, in_qsiz

2020-03-31 00:01:24,663 : INFO : EPOCH 5 - PROGRESS: at 63.43% examples, 100922 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:25,666 : INFO : EPOCH 5 - PROGRESS: at 63.69% examples, 100939 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:26,672 : INFO : EPOCH 5 - PROGRESS: at 63.95% examples, 100912 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:27,752 : INFO : EPOCH 5 - PROGRESS: at 64.25% examples, 100895 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:28,838 : INFO : EPOCH 5 - PROGRESS: at 64.55% examples, 100876 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:29,878 : INFO : EPOCH 5 - PROGRESS: at 64.79% examples, 100875 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:30,963 : INFO : EPOCH 5 - PROGRESS: at 65.08% examples, 100896 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:31,984 : INFO : EPOCH 5 - PROGRESS: at 65.35% examples, 100906 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:01:33,060 : INFO : EPOCH 5 - PROGRESS: at 65.70% examples, 100887 words/s, in_qsiz

2020-03-31 00:02:41,016 : INFO : EPOCH 5 - PROGRESS: at 83.80% examples, 100631 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:42,026 : INFO : EPOCH 5 - PROGRESS: at 84.05% examples, 100641 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:43,091 : INFO : EPOCH 5 - PROGRESS: at 84.33% examples, 100634 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:44,103 : INFO : EPOCH 5 - PROGRESS: at 84.61% examples, 100642 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:45,157 : INFO : EPOCH 5 - PROGRESS: at 84.90% examples, 100639 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:46,173 : INFO : EPOCH 5 - PROGRESS: at 85.17% examples, 100647 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:47,198 : INFO : EPOCH 5 - PROGRESS: at 85.44% examples, 100651 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:48,235 : INFO : EPOCH 5 - PROGRESS: at 85.71% examples, 100652 words/s, in_qsize 0, out_qsize 0
2020-03-31 00:02:49,275 : INFO : EPOCH 5 - PROGRESS: at 85.99% examples, 100653 words/s, in_qsiz

In [6]:
model.save(result_dir_path+'CORD-19-FT_200d_5w_5i_5mc.model')


2020-03-31 00:03:50,152 : INFO : saving FastText object under /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_5i_5mc.model, separately None
2020-03-31 00:03:50,153 : INFO : storing np array 'vectors' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_5i_5mc.model.wv.vectors.npy
2020-03-31 00:03:50,228 : INFO : storing np array 'vectors_vocab' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_5i_5mc.model.wv.vectors_vocab.npy
2020-03-31 00:03:50,299 : INFO : storing np array 'vectors_ngrams' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-FT_200d_5w_5i_5mc.model.wv.vectors_ngrams.npy
2020-03-31 00:04:04,371 : INFO : not storing attribute vectors_norm
2020-03-31 00:04:04,373 : INFO : not storing attribute vectors_vocab_norm
2020-03-31 00:04:04,373 : INFO : not storing attribute vectors_ngrams_norm
2020-03-31 00:04:04,374 : INFO : not storing attribute buckets_word
2020-03-31 00:04:04,375 : INFO : storing np array 'syn1neg' to /home/santosh

In [7]:
model.wv.most_similar('SARS-CoV-2', topn=10)

2020-03-31 00:04:50,164 : INFO : precomputing L2-norms of word weight vectors
2020-03-31 00:04:50,396 : INFO : precomputing L2-norms of ngram weight vectors


[('SARS-CoV-2-F2', 0.9803857207298279),
 ('SARS-CoV-2-F1', 0.9775040149688721),
 ('SARS-CoV-', 0.9563692212104797),
 ('2019-nCoV/SARS-CoV-2', 0.9542135000228882),
 ('SARS-CoV-2.', 0.9538172483444214),
 ('SARS-CoV-NC', 0.9490789175033569),
 ('SARS-CoV-ΔE', 0.9482947587966919),
 ('SARS-CoV-cpsR-19', 0.9448564648628235),
 ('SARS-CoV-ΔE-8a-dup', 0.9360930919647217),
 ('SARS-CoV-1', 0.9319473505020142)]

# Generate Paragraph Embeddings